# Imported Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
os.chdir("C:/Jupyter_Projects/Audible_Project")
cwd = os.getcwd()

In [3]:
df = pd.read_csv('audible_uncleaned.csv')

# Data Exploration

In [4]:
df

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,Writtenby:GeronimoStilton,Narratedby:BillLobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,Writtenby:RickRiordan,Narratedby:RobbieDaymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,Writtenby:JeffKinney,Narratedby:DanRussell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00
3,Daughter of the Deep,Writtenby:RickRiordan,Narratedby:SoneelaNankani,11 hrs and 16 mins,05-10-21,English,4.5 out of 5 stars12 ratings,615.00
4,"The Lightning Thief: Percy Jackson, Book 1",Writtenby:RickRiordan,Narratedby:JesseBernstein,10 hrs,13-01-10,English,4.5 out of 5 stars181 ratings,820.00
...,...,...,...,...,...,...,...,...
7823,Jeu d'évasion ou complot? [Escape Game or Cons...,Writtenby:CarolynChouinard,Narratedby:NicolasSavard-L'Herbier,2 hrs and 11 mins,30-03-22,french,Not rated yet,439.00
7824,Emotionen und andere Störfaktoren,Writtenby:KaiPannen,Narratedby:NilsKretschmer,2 hrs and 27 mins,01-04-22,german,Not rated yet,367.00
7825,Dampfloks gegen Dieselloks,"Writtenby:Mattel,BurkardMiltenberger",Narratedby:MontyArnold,16 mins,30-03-22,german,Not rated yet,99.00
7826,Squad Goals,Writtenby:ErikaJ.Kendrick,Narratedby:JonieceAbbott-Pratt,6 hrs and 30 mins,29-03-22,English,Not rated yet,703.00


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7828 entries, 0 to 7827
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         7828 non-null   object
 1   author       7828 non-null   object
 2   narrator     7828 non-null   object
 3   time         7828 non-null   object
 4   releasedate  7828 non-null   object
 5   language     7827 non-null   object
 6   stars        7827 non-null   object
 7   price        7827 non-null   object
dtypes: object(8)
memory usage: 489.4+ KB


### Duplicate check and missing data check

In [6]:
df.duplicated().sum()

0

In [7]:
df.isnull().sum().sum()

3

### All columns will have to be converted to proper data types (numeric, date, etc).

#### Entry 'You Are the Loveliest' has missing data.

### The stars column will have to be split to make a new column of numeric type for number of ratings.
### The original star column will be transformed to only contain the star rating.

# Fill in the missing data from Audible.com

In [8]:
df.at[7827,'price'] = '687'

In [9]:
df.at[7827,'stars'] = 'Not rated yet'

In [10]:
df.at[7827,'language'] = 'English'

In [11]:
df.at[7827,'releasedate'] = '29-03-22'

# Convert price column to integer data type

In [12]:
df['price'] = pd.to_numeric(df['price'], errors='coerce').fillna(0).astype(np.int32)

In [13]:
df['price'].head()

0    468
1    820
2    410
3    615
4    820
Name: price, dtype: int32

# Create new column 'ratings' using data from stars column

In [14]:
df['ratings'] = df['stars'].str.split(" ").str.get(-2).str.replace('stars', " ")

In [15]:
df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce').fillna(0).astype(np.int32)

In [16]:
df['ratings']

0        34
1        41
2        38
3        12
4       181
       ... 
7823      0
7824      0
7825      0
7826      0
7827      0
Name: ratings, Length: 7828, dtype: int32

# This method also works for 'stars'

In [17]:
df['stars'] = df['stars'].str.split(" ").str.get(0)

In [18]:
df['stars'] = pd.to_numeric(df['stars'], errors='coerce').fillna(0).astype(np.float32)

In [19]:
df['stars'] 

0       5.0
1       4.5
2       4.5
3       4.5
4       4.5
       ... 
7823    0.0
7824    0.0
7825    0.0
7826    0.0
7827    0.0
Name: stars, Length: 7828, dtype: float32

In [20]:
df['time'].head(1)

0    2 hrs and 20 mins
Name: time, dtype: object

# Time column will be split into two temporary columns and combined

In [21]:
hrs = df['time'].str.extract(r'(\d+(?= h.))').fillna(0).astype(np.int32) * 60
hrs.head(3)

,0
0,120
1,780
2,120


In [22]:
mins = df['time'].str.extract(r'(\d+(?= mi.))').fillna(0).astype(np.int32)
mins.head(3)

,0
0,20
1,8
2,3


In [23]:
df['time'] = hrs + mins
df['time'].head(3)

0    140
1    788
2    123
Name: time, dtype: int32

# Remove written and narrated by from author and narrator via replace

In [24]:
df['narrator'] = df['narrator'].str.replace('Narratedby:', '')

In [25]:
df['author'] = df['author'].str.replace('Writtenby:', '')

### Male author and narrator columns more readable

In [26]:
df['author'] = df['author'].str.replace(r'([a-z])([A-Z])', r'\1_\2',regex=True)
df['narrator'] = df['narrator'].str.replace(r'([a-z])([A-Z])', r'\1_\2',regex=True)
df.head()

,name,author,narrator,time,releasedate,language,stars,price,ratings
0,Geronimo Stilton #11 & #12,Geronimo_Stilton,Bill_Lobely,140,04-08-08,English,5.0,468,34
1,The Burning Maze,Rick_Riordan,Robbie_Daymond,788,01-05-18,English,4.5,820,41
2,The Deep End,Jeff_Kinney,Dan_Russell,123,06-11-20,English,4.5,410,38
3,Daughter of the Deep,Rick_Riordan,Soneela_Nankani,676,05-10-21,English,4.5,615,12
4,"The Lightning Thief: Percy Jackson, Book 1",Rick_Riordan,Jesse_Bernstein,600,13-01-10,English,4.5,820,181


### Convert remaining columns to proper data types

In [27]:
df['price'] = pd.to_numeric(df['price'].astype(np.int32))
df['stars'] = pd.to_numeric(df['stars'].astype(np.float32))
df['language'] =  df['language'].astype('string')
df['releasedate'] = pd.to_datetime(df['releasedate'], format='%d-%m-%y')
df['narrator'] =  df['narrator'].astype('string')
df['author'] =  df['author'].astype('string')
df['name'] =  df['name'].astype('string')

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7828 entries, 0 to 7827
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   name         7828 non-null   string        
 1   author       7828 non-null   string        
 2   narrator     7828 non-null   string        
 3   time         7828 non-null   int32         
 4   releasedate  7828 non-null   datetime64[ns]
 5   language     7828 non-null   string        
 6   stars        7828 non-null   float32       
 7   price        7828 non-null   int32         
 8   ratings      7828 non-null   int32         
dtypes: datetime64[ns](1), float32(1), int32(3), string(4)
memory usage: 428.2 KB


In [29]:
df

,name,author,narrator,time,releasedate,language,stars,price,ratings
0,Geronimo Stilton #11 & #12,Geronimo_Stilton,Bill_Lobely,140,2008-08-04,English,5.0,468,34
1,The Burning Maze,Rick_Riordan,Robbie_Daymond,788,2018-05-01,English,4.5,820,41
2,The Deep End,Jeff_Kinney,Dan_Russell,123,2020-11-06,English,4.5,410,38
3,Daughter of the Deep,Rick_Riordan,Soneela_Nankani,676,2021-10-05,English,4.5,615,12
4,"The Lightning Thief: Percy Jackson, Book 1",Rick_Riordan,Jesse_Bernstein,600,2010-01-13,English,4.5,820,181
...,...,...,...,...,...,...,...,...,...
7823,Jeu d'évasion ou complot? [Escape Game or Cons...,Carolyn_Chouinard,Nicolas_Savard-L'Herbier,131,2022-03-30,french,0.0,439,0
7824,Emotionen und andere Störfaktoren,Kai_Pannen,Nils_Kretschmer,147,2022-04-01,german,0.0,367,0
7825,Dampfloks gegen Dieselloks,"Mattel,Burkard_Miltenberger",Monty_Arnold,16,2022-03-30,german,0.0,99,0
7826,Squad Goals,Erika_J.Kendrick,Joniece_Abbott-Pratt,390,2022-03-29,English,0.0,703,0
